###API KEY

In [ ]:
import os

os.environ["SEC_API_USER_AGENT"] =

os.environ["BEA_API_KEY"] =

os.environ["FRED_API_KEY"] =

###Install


In [ ]:
!pip install finagg
!finagg install --help

In [1]:
from sqlalchemy import create_engine
import sqlite3
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


###Inspect Tables

In [2]:
from sqlalchemy import create_engine, inspect, MetaData

db_path = '/content/drive/MyDrive/Facultad/PFC/Implementacion/BD/finagg_data.sqlite'
# Combine all metadata
all_metadata = MetaData()
# Setup the database and create tables based on the combined metadata
engine = create_engine(f'sqlite:///{db_path}')
all_metadata.create_all(engine)

# Reflect the tables from the engine into the metadata
all_metadata.reflect(bind=engine)

# Get a list of all the tables
tables = all_metadata.tables.keys()

# Use an inspector to fetch columns and other details
inspector = inspect(engine)

#Ver datos
for table in tables:
    print("Table:", table)
    for column in inspector.get_columns(table):
        print("  Column:", column['name'], column['type'])

Table: fred.raw.series
  Column: series_id VARCHAR
  Column: realtime_start VARCHAR
  Column: realtime_end VARCHAR
  Column: date VARCHAR
  Column: value FLOAT
Table: fred.refined.economic
  Column: date VARCHAR
  Column: CIVPART FLOAT
  Column: LOG_CHANGE(CPIAUCNS) FLOAT
  Column: LOG_CHANGE(CSUSHPINSA) FLOAT
  Column: LOG_CHANGE(DJIA) FLOAT
  Column: FEDFUNDS FLOAT
  Column: LOG_CHANGE(GDP) FLOAT
  Column: LOG_CHANGE(GDPC1) FLOAT
  Column: GS10 FLOAT
  Column: LOG_CHANGE(M2) FLOAT
  Column: MICH FLOAT
  Column: LOG_CHANGE(NASDAQ100) FLOAT
  Column: LOG_CHANGE(NASDAQCOM) FLOAT
  Column: PSAVERT FLOAT
  Column: LOG_CHANGE(SP500) FLOAT
  Column: UMCSENT FLOAT
  Column: UNRATE FLOAT
  Column: LOG_CHANGE(WALCL) FLOAT
Table: fundam.refined.fundam
  Column: ticker VARCHAR
  Column: date VARCHAR
  Column: PriceBookRatio FLOAT
  Column: PriceEarningsRatio FLOAT
Table: sec.raw.submissions
  Column: cik VARCHAR
  Column: ticker VARCHAR
  Column: entity_type VARCHAR
  Column: sic VARCHAR
  Colum

###Number of Rows

In [3]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
for table in tables:
    table_name = table[0]
    # Colocar el nombre de la tabla entre comillas dobles
    cursor.execute(f'SELECT COUNT(*) FROM "{table_name}";')
    count = cursor.fetchone()[0]
    print(f"Table {table_name} has {count} rows.")

Table fred.raw.series has 34527 rows.
Table fred.refined.economic has 2353 rows.
Table indices.raw.djia has 30 rows.
Table indices.raw.sp500 has 503 rows.
Table indices.raw.nasdaq100 has 101 rows.
Table sec.raw.submissions has 471 rows.
Table sec.raw.tags has 172005 rows.
Table sec.refined.annual has 1276 rows.
Table sec.refined.annual.normalized has 664 rows.
Table sec.refined.quarterly has 3519 rows.
Table sec.refined.quarterly.normalized has 3113 rows.
Table yfinance.raw.prices has 4257432 rows.
Table yfinance.refined.daily has 4039449 rows.
Table fundam.refined.fundam has 283344 rows.
Table fundam.refined.fundam.normalized has 263607 rows.


###Tickers

In [4]:
db_path = '/content/drive/MyDrive/Facultad/PFC/Implementacion/BD/finagg_data.sqlite'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT DISTINCT ticker FROM 'fundam.refined.fundam'")
rows = cursor.fetchall()
for row in rows:
    print(row)
conn.close()

('A',)
('AAPL',)
('ALGN',)
('ALLE',)
('AMAT',)
('AMCR',)
('ANET',)
('APTV',)
('ATVI',)
('AVGO',)
('AXON',)
('CARR',)
('CDW',)
('CLX',)
('COO',)
('COST',)
('CPRT',)
('CRL',)
('CSCO',)
('CTLT',)
('CVS',)
('CZR',)
('D',)
('DD',)
('DVA',)
('DXCM',)
('EA',)
('ECL',)
('EIX',)
('ENPH',)
('EW',)
('FANG',)
('FE',)
('FSLR',)
('FTNT',)
('GE',)
('GEHC',)
('GM',)
('GNRC',)
('GOOG',)
('GPN',)
('HCA',)
('HD',)
('HII',)
('HLT',)
('HPE',)
('HSIC',)
('HWM',)
('IDXX',)
('INCY',)
('ISRG',)
('JNPR',)
('KDP',)
('KMI',)
('KVUE',)
('LCID',)
('LDOS',)
('LH',)
('LHX',)
('LOW',)
('LRCX',)
('LULU',)
('LVS',)
('LW',)
('MAS',)
('MDT',)
('MELI',)
('MGM',)
('MHK',)
('MLM',)
('MMM',)
('MPC',)
('MPWR',)
('MRNA',)
('MRO',)
('MSFT',)
('MTD',)
('MU',)
('NCLH',)
('NEE',)
('NOC',)
('NTAP',)
('ON',)
('OTIS',)
('PAYC',)
('PNR',)
('PODD',)
('POOL',)
('PPG',)
('PWR',)
('QCOM',)
('QRVO',)
('REGN',)
('RMD',)
('ROL',)
('RVTY',)
('SBUX',)
('SEE',)
('SIRI',)
('SJM',)
('SLB',)
('SNPS',)
('SPGI',)
('STE',)
('STLD',)
('STX',)
('SWKS',)

###Data Range

In [7]:
db_path = '/content/drive/MyDrive/Facultad/PFC/Implementacion/BD/finagg_data.sqlite'
# Combine all metadata
# Setup the database and create tables based on the combined metadata
engine = create_engine(f'sqlite:///{db_path}')

def get_date_range_from_table(engine, table_name, column_name, value):
    with engine.connect() as connection:
        query = text(f"""
            SELECT MIN(date), MAX(date)
            FROM "{table_name}"
            WHERE {column_name} = :value
        """)
        result = connection.execute(query, {"value": value}).fetchone()
        return result if result else (None, None)


def get_SEC_date_range_from_table(engine, table_name, column_name, value):
    with engine.connect() as connection:
        query = text(f"""
            SELECT MIN(fy), MAX(fy), MIN(fp), MAX(fp)
            FROM "{table_name}"
            WHERE {column_name} = :value
        """)
        result = connection.execute(query, {"value": value}).fetchone()
        if result:
            min_year, max_year, min_period, max_period = result
            start_date = f"{min_year}-{min_period}"
            end_date = f"{max_year}-{max_period}"
            return start_date, end_date
        else:
            return None, None

def get_fred_range(engine):
    with engine.connect() as connection:
        query = text(f"""
            SELECT MIN(date), MAX(date)
            FROM "fred.refined.economic"
        """)
        result = connection.execute(query).fetchone()
        return result if result else (None, None)

In [13]:
from sqlalchemy import text

ticker = "AAPL"  # Coloca el ticker que quieres consultar aquí
ticker_tables = ["yfinance.refined.daily", "fundam.refined.fundam", "fundam.refined.fundam.normalized"]  # tablas

cik_query = text("""
  SELECT cik
  FROM "sec.raw.submissions"
  WHERE ticker = :ticker
  LIMIT 1
""")
with engine.connect() as connection:
    cik = connection.execute(cik_query, {'ticker': ticker}).scalar()

cik_tables = ["sec.refined.quarterly.normalized"]  # Agrega las demás tablas que correspondan a 'cik'

#Macroeconomic data
start_date, end_date = get_fred_range(engine)
print(f"Table fred.refined.economic: Data range: {start_date} a {end_date}")

# Obtener rangos de fechas para tablas basadas en ticker
print(f"Empresa: {ticker}")
for table in ticker_tables:
  start_date, end_date = get_date_range_from_table(engine, table, "ticker", ticker)
  print(f"Table {table}: Data range: {start_date} to {end_date}")

# Obtener rangos de fechas para tablas basadas en cik
start_date, end_date = get_SEC_date_range_from_table(engine, "sec.refined.quarterly.normalized", "cik", cik)
print(f"Table sec.refined.quarterly.normalized: Data range: {start_date} to {end_date}")


Table fred.refined.economic: Data range: 2014-10-02 a 2023-08-28
Empresa: AAPL
Table yfinance.refined.daily: Data range: 1980-12-15 to 2023-08-29
Table fundam.refined.fundam: Data range: 2010-01-25 to 2023-08-29
Table fundam.refined.fundam.normalized: Data range: 2010-01-25 to 2023-08-29
Table sec.refined.quarterly.normalized: Data range: 2010-Q1 to 2023-Q3


###Data SEC for Specific Ticker

In [11]:
def get_data_sec_ticker(engine, ticker):
    cik_query = text("""
      SELECT cik
      FROM "sec.raw.submissions"
      WHERE ticker = :ticker
      LIMIT 1
    """)
    with engine.connect() as connection:
        value_cik = connection.execute(cik_query, {'ticker': ticker}).scalar()

    with engine.connect() as connection:
        query = text(f"""
            SELECT *
            FROM "sec.refined.quarterly.normalized"
            WHERE cik = :value
        """)
        result = connection.execute(query, {"value": value_cik}).fetchall()
        return result if result else None

ticker = "AAPL"
data = get_data_sec_ticker(engine,ticker)
for d in data:
  print(d)

('0000320193', '2010-04-21', 2010, 'Q2', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.148114498605753, 1.116804761674308, -1.0592892958144877, 1.1497151530344076, 0.1932402368269617, 1.1029110500380166, 1.0077607934963866, 0.07841225492376207)
('0000320193', '2010-07-21', 2010, 'Q3', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.148114498605753, 1.116804761674308, -1.0592892958144877, 1.1526388605804878, 0.1932402368269617, 1.0933780620536453, 1.0158098452706459, 0.07841225492376207)
('0000320193', '2011-01-19', 2011, 'Q1', 0.9788155052042176, 0.0740318129444878, 0.4072817124119062, 1.1047898602862558, 1.1546712923570324, 1.134775548600791, 0.6525393918175677, 0.47603194316240766, 1.1103037237090845, -0.5215821085108953, 1.1104099517021981, -0.7096570030023399, 0.7968840245990643, 0.856600423006808, -0.9905408749828999)
('0000320193', '2011-04-21', 2011, 'Q2', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.2506653352720475, 1.4484658203149217, -0.10723296664142487, 1.3822426943194839, -1.001336015599892, 0.6511